In [1]:
import os
from IPython.display import clear_output
os.chdir('..')

In [2]:
os.environ["CONFIG"] = "configs/small/small_p_fold_tf.yaml"

In [3]:
%%bash
cd data
chmod +x get_dataset.sh
./get_dataset.sh

mkdir: cannot create directory ‘raw’: File exists


In [4]:
!python3 data/prepare_data.py --config_path $CONFIG \
    --sec_req data/raw/SecReq --promise data/raw/nfr/nfr.arff \
    --concord data/raw/NFRClassifier/gate/application-resources/Requirements/ \
    --cchit data/raw/CCHIT.xls --owasp data/raw/OWASP -o data/processed

In [5]:
from constants import DATASETS_CONFIG_PATH

pfold = [[3, 9, 11], [1, 5, 12], [6, 10, 13], [1, 8, 14], [3, 12, 15], [2, 5, 11], [6, 9, 14], [7, 8, 13], [2, 4, 15], [4, 7, 10]]

all_files = list(range(1, 16))

for test_files in pfold:
  train_files = [str(j) for j in all_files if j not in test_files]
  test_files = list(map(str, test_files))
  print(train_files, test_files)
  train_files_config = "\n  ".join([
    f"    - {file_}" for file_ in train_files
  ])
  test_files_config = "\n  ".join([
    f"    - {file_}" for file_ in test_files
  ])
  configuration = f"""
  train_datasets:
    promise:
  {train_files_config}
  valid_datasets:
    promise:
  {test_files_config}
"""

  with open(DATASETS_CONFIG_PATH, "w+") as f:
    f.write(configuration)

  !CUBLAS_WORKSPACE_CONFIG=:4096:8 python3 train.py $CONFIG


9259, 'eval_recall': 0.30120481927710846, 'eval_runtime': 0.5684, 'eval_samples_per_second': 218.148, 'epoch': 11.0}
['2', '4', '15']
 56%|████████████████████████▊                   | 9/16 [00:00<00:00, 33.83it/s]

{'eval_loss': 5.545562744140625, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.5761, 'eval_samples_per_second': 215.235, 'epoch': 1.0}
 81%|██████████████████████████████████▉        | 13/16 [00:00<00:00, 32.31it/s]/home/hawk/anaconda3/envs/thesis/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

{'eval_loss': 2.255593776702881, 'eval_accuracy': 0.33064516129032256, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.5826, 'eval_samples_per_second': 212.843, 